# LangGraph

## Download LLM

In [ ]:
from os import makedirs, environ
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from huggingface_hub import login
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from time import time

_ = load_dotenv(dotenv_path=".env", override=True)
login(token=os.environ["HF_TOKEN"])

model_name = "google/gemma-3-1b-it"
local_path = "./models"
makedirs(local_path, exist_ok=True)

start_time = time()
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    token=environ["HF_TOKEN"],
    device_map="auto", dtype=torch.bfloat16
)
model.save_pretrained(save_directory=local_path)

tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name,
    token=environ["HF_TOKEN"]
)
tokenizer.save_pretrained(save_directory=local_path)
end_time = time() - start_time

print("Time taken: %.2f"%(end_time))

## Import LLM

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

local_dir = "./models"
llm = HuggingFacePipeline.from_model_id(
    task="text-generation", model_id=local_dir,
    pipeline_kwargs={"max_new_tokens": 512, "temperature": 0.1})
llm = ChatHuggingFace(llm=llm)

### State Schema

In [ ]:
from typing import TypedDict

class AgentState(TypedDict):
    """
    Represents the state of agent.
    Attr:
        messages: A list of messages making up the conversation.
        user_query: The initial query from the user.
        tool_output: Any output from tools used by the agent.
    """
    

### Chatbot Agent

In [4]:
from langchain_core.prompts import ChatPromptTemplate

# Create a simple prompt template
prompt = ChatPromptTemplate.from_messages(messages=[
    ("system", "You are a helpful AI assistant. Always introduce yourself."),
    ("user", "{input}")
])

# Combine the prompt and LLM into a simple chain
chain = prompt | llm

# Let's ask agent a question!
response = chain.invoke(input={"input": "What is SWIFT financial messages?"})

print("AI Agent says:\n")
print(response.content)

AI Agent says:

<bos><start_of_turn>user
You are a helpful AI assistant. Always introduce yourself.

What is SWIFT financial messages?<end_of_turn>
<start_of_turn>model
Hello there! I'm Gemma, a large language model created by the Gemma team at Google DeepMind. I'm here to help you with a wide range of tasks, and I'm particularly good at understanding and responding to questions about finance and related topics. 

**SWIFT financial messages** are a standardized system for transmitting financial messages between banks. Think of them as a digital postal service for the financial world. Here's a breakdown of what they are:

* **What they do:** SWIFT (Society for Worldwide Interbank Financial Telecommunication) is a global network that allows banks to securely exchange information about payments, such as transfers of funds.
* **How they work:** They use a standardized protocol (called SWIFT Standard Code) to format messages, ensuring they are understood by banks worldwide. These messages c

In [ ]:
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

# Define a simple in-meory chat history store
class ChatMessageHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []

    def add_user_message(self, message: str) -> None:
        self.messages.append(HumanMessage(content=message))

    def add_ai_message(self, message: str) -> None:
        self.messages.append(AIMessage(content=message))

    def clear(self) -> None:
        self.messages = []

    @property
    def messages(self) -> list[BaseMessage]:
        return self.messages

    @messages.setter
    def messages(self, value: list[BaseMessage]) -> None:
        self._messages = value

# Create a place to store conversation history
store = {}
def get_session_history(session_id: str) -> ConversationBufferMemory:
    if session_id not in store:
        store[session_id] = ConversationBufferMemory(
            chat_memory=ChatMessageHistory(),
            return_messages=True,
            memory_key="chat_history"
        )
    return store[session_id]

prompt_with_history = ChatPromptTemplate.from_messages(messages=[
    ("system", "You're a helpful AI assistant. Always remember past conversations."),
    ("placeholder", "{chat_history}"), # This is where the memory is inserted
    ("user", "{input}")
])

# Combine the new prompt and LLM into a chain
chat_with_history = prompt_with_history | llm

# Now integrate with RunnableWithMessageHistory
conversational_chain = RunnableWithMessageHistory(
    runnable=chat_with_history,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

# We need to provide a session_id to store the history
config = {"configurable": {"session_id": "my_first_session"}}

response1 = conversational_chain.invoke(config=config,
    input={"input": "Hello, I'm Meng"})
print(f"AI Agent says: {response1.content}")

response2 = conversational_chain.invoke(config=config,
    input={"input": "What's my name?"})
print(f"AI Agent says: {response2.content}")